### This notebook allows the user to create a directory structure and locally upload the metadata to the Metadata Evaluation repository.

In [1]:
import pandas as pd
import os
from os import walk
import shutil
from ipywidgets import *
import ipywidgets as widgets
import requests
import io
from IPython.display import display
import metadataEvaluation

### Now let's select some metadata.

In [2]:
#functions to identify and label an xml collection for Metadata Evaluation
def OrganizationChoices(organization):
    
    metadataEvaluation.Organization=organization
    print("Curating Organization", metadataEvaluation.Organization)
    
def CollectionName(collection):
    metadataEvaluation.Collection=collection
    print("Collection is named", metadataEvaluation.Collection)
    
def DialectName(dialect):
    metadataEvaluation.Dialect=dialect
    print("Collection is written in the", metadataEvaluation.Dialect,"dialect")   
    
def metadataLocation(directory):
    metadataEvaluation.MetadataLocation=directory
    print("The local directory", metadataEvaluation.MetadataLocation,"contains the metadata for evaluation")    
    
w=interactive(OrganizationChoices, organization='')    
w2=interactive(CollectionName, collection='')
w3=interactive(DialectName, dialect='')
w4=interactive(metadataLocation, directory='')    

display(w)
display(w2)
display(w3)
display(w4)

Evaluate the collection

In [8]:
%cd ../zip
MetadataDestination=os.path.join(metadataEvaluation.Organization,metadataEvaluation.Collection,metadataEvaluation.Dialect,'xml')
os.makedirs(MetadataDestination, exist_ok=True)
os.makedirs(os.path.join('../data',metadataEvaluation.Organization), exist_ok=True)
src_files = os.listdir(metadataEvaluation.MetadataLocation)
for file_name in src_files:
    full_file_name = os.path.join(metadataEvaluation.MetadataLocation, file_name)
    if (os.path.isfile(full_file_name)):
        shutil.copy(full_file_name, MetadataDestination)
shutil.make_archive('../upload/metadata', 'zip', os.getcwd())

%cd ../upload 
# Send metadata package, read the response into a dataframe
url = 'http://metadig.nceas.ucsb.edu/metadata/evaluator'
files = {'zipxml': open('metadata.zip', 'rb')}
r = requests.post(url, files=files, headers={"Accept-Encoding": "gzip"})
r.raise_for_status()
EvaluatedMetadataDF = pd.read_csv(io.StringIO(r.text), quotechar='"')

#Change directories, delete upload directory and zip. Delete copied metadata.
%cd ../
shutil.rmtree('upload')
%cd zip
shutil.rmtree(metadataEvaluation.Organization)
%cd ../data

metadataEvaluation.dataProducts(EvaluatedMetadataDF)

/Users/scgordon/MetadataEvaluation/zip
/Users/scgordon/MetadataEvaluation/upload
/Users/scgordon/MetadataEvaluation
/Users/scgordon/MetadataEvaluation/zip
/Users/scgordon/MetadataEvaluation/data
No elements contain a variant of 'Not provided' in their content for this collection
Data products created for the 2008 collection from the LTER organization


In [46]:
#Create a list of subdirectories in the collection directory of MILE2 to select metadata for evaluation
DataDirectory = "../data/"+metadataEvaluation.Organization
DataProducts = os.listdir(DataDirectory)

#Create a function to select the organization the metadata comes from
    
def DataProductChoices(dataProduct):
    global sEvaluatedMetadataDF
    DataDirectory = "../data/"+metadataEvaluation.Organization
    filePath=DataDirectory+'/'+dataProduct
    sEvaluatedMetadataDF= pd.read_csv(os.path.join(filePath))
    return sEvaluatedMetadataDF    

#organization selector dropdown    
interactive(DataProductChoices,dataProduct=sorted(DataProducts)) 


In [47]:
sEvaluatedMetadataDF

,XPath,Collection,XPathCount,RecordCount,AverageOccurrencePerRecord,CollectionOccurrence%
0,Number of Records,module_module,18,18,1,100.00%
1,/acquisitionInformation/instrument,module_module,53,18,3,100.00%
2,/acquisitionInformation/instrument/@id,module_module,65,18,4,100.00%
3,/acquisitionInformation/instrument/citation/da...,module_module,18,18,1,100.00%
4,/acquisitionInformation/instrument/citation/title,module_module,106,18,6,100.00%
5,/acquisitionInformation/instrument/description...,module_module,18,18,1,100.00%
6,/acquisitionInformation/instrument/identifier/...,module_module,159,18,9,100.00%
7,/acquisitionInformation/instrument/identifier/...,module_module,53,18,3,100.00%
8,/acquisitionInformation/instrument/mountedOn/@...,module_module,64,18,4,100.00%
9,/acquisitionInformation/instrument/sensor/@id,module_module,65,18,4,100.00%


### Combine different types of evaluated data to compare between collections or observe trends over time

* [Combine_Data](Combine_Data.ipynb)
